## LSTM model Engagement Recognition (5 classes) + 25 body keypoints

In [3]:
from keras.layers import Dense, LSTM, GRU, Dropout, Flatten, Embedding,  BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras.models import Model,Sequential
from keras.utils import to_categorical
from keras import regularizers
from keras.optimizers import SGD
from clr_callback import *
%matplotlib inline
import time
import pickle
import os
import math
import torch.nn as nn
from torch.nn.modules.utils import _triple
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report



In [4]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
SPLIT_METHOD_child = "CHILD"
SPLIT_METHOD_session = "SESSION"
SPLIT_METHOD_random = "RANDOM"

# WITH FACE, BODY, HANDS (411) =>
FEAT_NUM_all = 201

# WITH BODY AND HANDS (201) =>
FEAT_NUM_body = (25 + 21 + 21) * 3

# WITH FACE ONLY (210) =>
FEAT_NUM_face = 70 * 3

SEQ_LENGTH = 1

NB_CLASS = 6
CSV_FILE = "ASD_36_res1.csv"
VARS_DIR = "var"


In [6]:
import pandas
data = pandas.read_csv('ASD_36_res1.csv')

In [7]:
X = data.iloc[:, 3:78]
y = data['engagement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=1)

In [8]:
X_train.shape


(253124, 75)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [10]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(227811, 75)
(25313, 75)
(41207, 75)


In [11]:
X_train = X_train.to_numpy()
X_test =X_test.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

In [12]:
type(X_train)

numpy.ndarray

In [13]:

early_stop = EarlyStopping(monitor='loss', patience=2)


In [14]:
xx = tf.convert_to_tensor(X_train.reshape(227811,1 , 75))
xv = tf.convert_to_tensor(X_val.reshape(25313, 1, 75))
xt = tf.convert_to_tensor(X_test.reshape(41207,1, 75))
yy = tf.convert_to_tensor(y_train.reshape(227811, 1))
yv = tf.convert_to_tensor(y_val.reshape(25313, 1))
yt = tf.convert_to_tensor(y_test.reshape(41207, 1))

In [20]:
yy= to_categorical(y_train)
yv =  to_categorical(y_val)
yt = to_categorical(y_train)

In [34]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation='relu', input_shape=(75,)))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(512, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 128)               9728      
_________________________________________________________________
dense_22 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_23 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_24 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_25 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_26 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_27 (Dense)             (None, 256)              

In [35]:
from keras.optimizers import SGD
opt = "adam"#SGD(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [36]:
hist = model.fit(x = X_train , y = yy, epochs=20, validation_data =(X_val, yv), batch_size= 64)


Epoch 1/20
3560/3560 [==============================] - 7s 2ms/step - loss: 1.4215 - accuracy: 0.4614 - val_loss: 1.3749 - val_accuracy: 0.4673
Epoch 2/20
3560/3560 [==============================] - 7s 2ms/step - loss: 1.3379 - accuracy: 0.4832 - val_loss: 1.3299 - val_accuracy: 0.4860
Epoch 3/20
3560/3560 [==============================] - 7s 2ms/step - loss: 1.2994 - accuracy: 0.4956 - val_loss: 1.3192 - val_accuracy: 0.4885
Epoch 4/20
3560/3560 [==============================] - 7s 2ms/step - loss: 1.2690 - accuracy: 0.5091 - val_loss: 1.2889 - val_accuracy: 0.5048
Epoch 5/20
3560/3560 [==============================] - 7s 2ms/step - loss: 1.2457 - accuracy: 0.5198 - val_loss: 1.2583 - val_accuracy: 0.5163
Epoch 6/20
3560/3560 [==============================] - 7s 2ms/step - loss: 1.2261 - accuracy: 0.5277 - val_loss: 1.2340 - val_accuracy: 0.5246
Epoch 7/20
3560/3560 [==============================] - 7s 2ms/step - loss: 1.2061 - accuracy: 0.5366 - val_loss: 1.2210 - val_accuracy:

In [37]:
h = model.evaluate(X_train,yt)

7120/7120 [==============================] - 8s 1ms/step - loss: 1.0135 - accuracy: 0.6192
